In [3]:
# code to produce openmax results for DeepStreamCE setup 01-6 (airplane, automobile with frog as the concept evolution class)
# the setup can be changed by changing/adding setup numbers to the 'setups' array below

from IPython.display import clear_output
from __future__ import division
from openmax_compare import *
from openmax_compare_utils import *
from datetime import datetime
import time
import os
np.random.seed(12345)

class UnseenData:
    def __init__(self, id, instance, correctResult, discrepancyName):
        self.id = id
        self.instance = instance
        self.correctResult = correctResult
        self.discrepancyName = discrepancyName
        self.softmaxPredict = 0
        self.openmaxPredict = 0
        self.finalResult = 0

cwd = os.getcwd()
modelsDir = cwd.replace('openmax_compare','models')

alpha_ranks = [2]
tail_sizes = [9]
setups = [1]  # change/add (data setups 1 to 8) to this array. Data setups relating to the numbers are defined below.
useAllInstances = False  # change to false to use the number of instances defined in 'numInstances'
numInstances = 50   # The number of non-discrepancy instances and the number of concept evolution instances.  Used if 'useAllInstances' is set to False

for a in alpha_ranks:
    ALPHA_RANK = a
    for t in tail_sizes:
        TAIL_SIZE = t
        for s in setups:
            clear_output()
            setup = s

            ## Load already trained VGG16 keras model
            print('Loading model...')
            if setup == 1:
                ND_CLASSES = [0,1]
                CE_CLASS = 6
                loadData(ND_CLASSES, CE_CLASS)
                model = load_model(modelsDir + '/vgg16_cifar10_2classes01.h5')
            elif setup == 2:
                ND_CLASSES = [8,9]
                CE_CLASS = 3
                loadData(ND_CLASSES, CE_CLASS)
                model = load_model(modelsDir + '/vgg16_cifar10_2classes89.h5')
            elif setup == 3:
                ND_CLASSES = [0,9]
                CE_CLASS = 4
                loadData(ND_CLASSES, CE_CLASS)
                model = load_model(modelsDir + '/models/vgg16_cifar10_2classes09.h5')
            elif setup == 4:
                ND_CLASSES = [8,9]
                CE_CLASS = 2
                loadData(ND_CLASSES, CE_CLASS)
                model = load_model(modelsDir + '/vgg16_cifar10_2classes89.h5')
            elif setup == 5:
                ND_CLASSES = [0,8]
                CE_CLASS = 2
                loadData(ND_CLASSES, CE_CLASS)
                model = load_model(modelsDir + '/vgg16_cifar10_2classes08.h5')
            elif setup == 6:
                ND_CLASSES = [8,9]
                CE_CLASS = 1
                loadData(ND_CLASSES, CE_CLASS)
                model = load_model(modelsDir + '/vgg16_cifar10_2classes89.h5')
            elif setup == 7:
                ND_CLASSES = [3,6]
                CE_CLASS = 4
                loadData(ND_CLASSES, CE_CLASS)
                model = load_model(modelsDir + '/vgg16_cifar10_2classes36.h5')
            elif setup == 8:
                ND_CLASSES = [3,4]
                CE_CLASS = 7
                loadData(ND_CLASSES, CE_CLASS)
                model = load_model(modelsDir + 'models/vgg16_cifar10_2classes34.h5')

            print('Creating mean activation vector (MAV) and Weibull fit model for each class...')
            # Create mean activation vector (MAV) and weibull fit model
            # This function will create numpy files saving the MAV for future use
            create_model(model)
            print('Model created')

            print('Collecting test data...')
            # Compute softmax output for CIFAR-10 (n Classes) dataset. This model was trained on this dataset
            # CIFAR-10 (n Classes) already loaded in the first cell
            random = False
            displayFullResults = False
            x_train_nd, y_train_nd, x_test_nd, y_test_nd = getNdData()
            x_train_ce, y_train_ce, x_test_ce, y_test_ce = getCeData()

            if useAllInstances:
                numND = len(x_test_nd)
                numCE = len(x_test_ce)
            else:
                numND = numInstances
                numCE = numInstances
            
            dataList = []

            # get data the DNN was trained on
            for i in range(numND):
                if random:
                    index = np.random.randint(0,len(x_test_nd))
                else:
                    index = i
                test_x1 = x_test_nd[index]
                test_y1 = y_test_nd[index]
                data = UnseenData(index, test_x1, test_y1[0], 'ND')
                dataList.append(data)

            # get concept evolution data
            for i in range(numCE):
                if random:
                    index = np.random.randint(0,len(x_test_ce))
                else:
                    index = i
                test_x1_ce = x_test_ce[index]
                test_y1_ce = y_test_ce[index]
                data = UnseenData(index, test_x1_ce, test_y1_ce[0], 'CE')
                dataList.append(data)  

            startTime = time.time()
            print('Processing test data...')
            count = 0
            for i in dataList:       
                # Compute fc8 activation for the given image
                activation = compute_activation(model,i.instance)

                # Compute openmax activation
                softmax,openmax = compute_openmax(model,activation, ALPHA_RANK, TAIL_SIZE)
                i.softmaxPredict = softmax
                
                #i.openmaxPredict = openmax
                i.openmaxPredict = 1 - openmax
                if i.openmaxPredict == -1 or i.openmaxPredict == 2:
                    i.openmaxPredict = 'unknown'

                if displayFullResults:
                    image_show(i.instance,i.correctResult)
                    print ('Actual Label: ', i.correctResult)
                    print ('Prediction Softmax: ', i.softmaxPredict)
                    print ('Prediction openmax: ',i.openmaxPredict)
                else:
                    if count % 50 == 0:
                        print('processing instance ' + str(count) + ' of ' + str(len(dataList)))
                count += 1
            print('Data processed')  
            endTime = time.time()
            print(str(numND + numCE) + ' instances, total run time (s): ' + str(endTime - startTime))
            timePerInst =  str((endTime - startTime)/(numND + numCE))
            print('time per instance: ' + timePerInst)


            # DeepStreamCE calculations for DeepStreamCE definitions of TP, FP, TN, FN  
            # This is different from the TP, FP, TN, FN definitions in the OpenMax paper
            tp = 0
            fp = 0
            tn = 0
            fn = 0
            precision = 0
            recall = 0
            fMeasure = 0
            accuracy = 0
            for i in dataList:
                if i.discrepancyName == 'CE' and i.openmaxPredict == 'unknown':
                    i.finalResult = 'OUTLIER'
                    tp += 1
                if i.discrepancyName == 'ND' and i.openmaxPredict == 'unknown':
                    i.finalResult = 'OUTLIER'
                    fp += 1
                if i.discrepancyName == 'ND' and i.openmaxPredict != 'unknown':
                    i.finalResult = 'NOT_OUTLIER'
                    tn += 1
                if i.discrepancyName == 'CE' and i.openmaxPredict != 'unknown':
                    i.finalResult = 'NOT_OUTLIER'
                    fn += 1

            if tp+fp != 0:
                precision = tp/(tp+fp) # ratio of correctly identified CE's to all outliers
            if tp+fn != 0:
                recall = tp/(tp+fn)    # ratio of correctly identified CE's to all CE's
            if precision + recall != 0:
                fMeasure = 2*precision*recall/(precision + recall)
            if tp+fp+tn+fn != 0:
                accuracy = (tp+tn)/(tp+fp+tn+fn) # accuracy is is not a good indicator of performance when the class distribution is imbalanced

            # save results to csv
            print('Creating results')  
            csvLines = []
            csvLines.append(['alpha', 'tail','time per inst','',''])
            csvLines.append([ALPHA_RANK, TAIL_SIZE,timePerInst,'',''])
            csvLines.append(['TP (CE-OUTLIER)', 'FP (ND-OUTLIER)', 'TN (ND-NOT_OUTLIER)', 'FN (CE-NOT_OUTLIER)',''])
            csvLines.append([tp, fp, tn, fn,''])
            csvLines.append(['Accuracy','Precision', 'Recall', 'F-Measure',''])
            csvLines.append([accuracy, precision, recall, fMeasure, ''])
            csvLines.append(['','','','',''])
            csvLines.append(['ID','Instance=class' + str(CE_CLASS),'class','outlier','actual'])
            [csvLines.append([x.id,x.discrepancyName, x.openmaxPredict, x.finalResult, x.correctResult]) for x in dataList]
            now = datetime.now().strftime("%y%m%d_%H%M%S")
            ndClasses = "".join(map(str,ND_CLASSES))
            saveToCsv('output/openmax_%s_%s_DS--%s_results.csv'%(ndClasses,CE_CLASS, now), csvLines)
            print('Results csv file created')
            

Loading model...
Creating mean activation vector (MAV) and Weibull fit model for each class...
0 (4981, 32, 32, 3)
1 (4980, 32, 32, 3)
Model created
Processing test data...
processing instance 0 of 100
processing instance 50 of 100
Data processed
100 instances, total run time (s): 24.7814731598
time per instance: 0.247814731598
Creating results
Results csv file created
